In [0]:
%run ./_resources/00-setup $reset_all_data=false


# Delta Lake Uniform: Universal Format for all your Lakehouse tools


<img src="https://cms.databricks.com/sites/default/files/inline-images/image1_5.png" width="700px" style="float: right; margin-left: 50px"/>

Delta Universal Format (UniForm) automatically unifies table formats, without creating additional copies of data or more data silos. UniForm takes advantage of the fact that all three open lakehouse formats are thin layers of metadata atop Parquet data files. As writes are made, UniForm will incrementally generate this layer of metadata to spec for Hudi, Iceberg and Delta.

Teams that use query engines designed to work with Iceberg or Hudi data will be able to read Delta tables seamlessly, without having to copy data over or convert it. 

Customers don’t have to choose a single format, because **tables written by Delta will be universally accessible by Iceberg and Hudi readers.**


<!-- Collect usage data (view). Remove it to disable collection. View README for more details.  -->
<img width="1px" src="https://ppxrzfxige.execute-api.us-west-2.amazonaws.com/v1/analytics?category=data-engineering&org_id=3759185753378633&notebook=%2F03-Delta-Lake-Uniform&demo_name=delta-lake&event=VIEW&path=%2F_dbdemos%2Fdata-engineering%2Fdelta-lake%2F03-Delta-Lake-Uniform&version=1">
<!-- [metadata={"description":"Quick introduction to Delta Lake. <br/><i>Use this content for quick Delta demo.</i>",
 "authors":["quentin.ambard@databricks.com"],
 "db_resources":{}}] -->

In [0]:
CREATE TABLE IF NOT EXISTS user_uniform (
  id BIGINT,
  firstname STRING,
  lastname STRING,
  email STRING
)
TBLPROPERTIES ('delta.universalFormat.enabledFormats' = 'iceberg',
               'delta.enableIcebergCompatV2' = 'true')

In [0]:
INSERT INTO user_uniform 
  SELECT id, firstname, lastname, email 
  FROM user_delta;  

SELECT * FROM user_uniform;

Because we enabled `iceberg` as format, each subsequential write will update the Delta and Iceberg metadata. Your table now contains 2 metadata folders:

* `delta_log` containing all Delta Lake format metadata.
* `metadata` containing Iceberg format metadata.

In [0]:
SHOW TBLPROPERTIES user_uniform;

### Access your Delta Lake table using Iceberg REST Catalog API:

Our Delta Lake table is now available by any system reading Iceberg tables, such as native Iceberg reader or external system like Big Query.

In [0]:
%python
from databricks.sdk import WorkspaceClient
ws = WorkspaceClient()

table_info = ws.api_client.do('GET', f'/api/2.1/unity-catalog/tables/{catalog}.{schema}.user_uniform')
table_info['delta_uniform_iceberg']

To read a managed table, you can leverage the Iceberg Catalog:

```
curl -X GET -H "Authentication: Bearer $OAUTH_TOKEN" -H "Accept: application/json" \
https://<workspace-instance>/api/2.1/unity-catalog/iceberg/v1/catalogs/<uc_catalog_name>/namespaces/<uc_schema_name>/tables/<uc_table_name>
```

You should then receive a response like this:

```
{
  "metadata-location": "s3://bucket/path/to/iceberg/table/metadata/file",
  "metadata": <iceberg-table-metadata-json>,
  "config": {
    "expires-at-ms": "<epoch-ts-in-millis>",
    "s3.access-key-id": "<temporary-s3-access-key-id>",
    "s3.session-token":"<temporary-s3-session-token>",
    "s3.secret-access-key":"<temporary-secret-access-key>",
    "client.region":"<aws-bucket-region-for-metadata-location>"
  }
}
```

For more details, see [the Uniform Documentation](https://docs.databricks.com/en/delta/uniform.html)